In [ ]:
import os
import numpy as np
import cv2 as cv
from matplotlib import pyplot as plt
import csv
import pandas as pd

src = '/kaggle/input/image-matching-challenge-2022/'

test_samples = []
with open(f'{src}/test.csv') as f:
    reader = csv.reader(f, delimiter=',')
    for i, row in enumerate(reader):
        # Skip header.
        if i == 0:
            continue
        test_samples += [row]

In [ ]:
def example(batch_id, distance = 0.75, i = -1):
    
    df = pd.read_csv(f'{src}/train/{batch_id}/pair_covisibility.csv')
    #row = df.iloc[0]
    if i == -1:
        i = np.random.randint(len(df))
    print(i)
    row = df.iloc[i]
    F_true = row['fundamental_matrix']
    F_true = [float(i) for i in F_true.split(' ')]
    F_true = [a/F_true[-1] for a in F_true ]
    img_id1, img_id2 = row['pair'].split('-')

    img1 = cv.imread(f'{src}/train/{batch_id}/images/{img_id1}.jpg', 0)
    img2 = cv.imread(f'{src}/train/{batch_id}/images/{img_id2}.jpg', 0)
    
    sift = cv.SIFT_create()
    # find the keypoints and descriptors with SIFT
    kp1, des1 = sift.detectAndCompute(img1,None)
    kp2, des2 = sift.detectAndCompute(img2,None)
    # FLANN parameters
    FLANN_INDEX_KDTREE = 1
    index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 220) # trees = 500 and checks=500 gives a stable result
    search_params = dict(checks=220)
    flann = cv.FlannBasedMatcher(index_params,search_params)
    matches = flann.knnMatch(des1,des2,k=2)
    pts1 = []
    pts2 = []
    # ratio test as per Lowe's paper
    for i,(m,n) in enumerate(matches):
        if m.distance < distance *n.distance:  ############# 0.65 - 0.75 seems to work impressive
            pts2.append(kp2[m.trainIdx].pt)
            pts1.append(kp1[m.queryIdx].pt)

    pts1 = np.int32(pts1)
    pts2 = np.int32(pts2)
    F, mask = cv.findFundamentalMat(pts1,pts2,cv.FM_LMEDS)
    #print(F)
    
    # We select only inlier points
    pts1 = pts1[mask.ravel()==1]
    pts2 = pts2[mask.ravel()==1]
    
    def drawlines(img1,img2,lines,pts1,pts2):
        ''' img1 - image on which we draw the epilines for the points in img2
            lines - corresponding epilines '''
        r,c = img1.shape
        img1 = cv.cvtColor(img1,cv.COLOR_GRAY2BGR)
        img2 = cv.cvtColor(img2,cv.COLOR_GRAY2BGR)
        for r,pt1,pt2 in zip(lines,pts1,pts2):
            color = tuple(np.random.randint(0,255,3).tolist())
            x0,y0 = map(int, [0, -r[2]/r[1] ])
            x1,y1 = map(int, [c, -(r[2]+r[0]*c)/r[1] ])
            img1 = cv.line(img1, (x0,y0), (x1,y1), color,1)
            img1 = cv.circle(img1,tuple(pt1),5,color,-1)
            img2 = cv.circle(img2,tuple(pt2),5,color,-1)
        return img1,img2

    # Find epilines corresponding to points in right image (second image) and
    # drawing its lines on left image
    lines1 = cv.computeCorrespondEpilines(pts2.reshape(-1,1,2), 2,F)
    lines1 = lines1.reshape(-1,3)
    img5,img6 = drawlines(img1,img2,lines1,pts1,pts2)
    # Find epilines corresponding to points in left image (first image) and
    # drawing its lines on right image
    lines2 = cv.computeCorrespondEpilines(pts1.reshape(-1,1,2), 1,F)
    lines2 = lines2.reshape(-1,3)
    img3,img4 = drawlines(img2,img1,lines2,pts2,pts1)
    plt.figure(figsize=(18,10))
    plt.subplot(121),plt.imshow(img5)
    plt.subplot(122),plt.imshow(img3)
    plt.show()
    return np.array(F_true), F.flatten()

In [ ]:
def run_experiment(distance = 0.75, i = -1):
    try:
        F_true, F_predicted = example(batch_id = 'brandenburg_gate', distance = distance, i = i)
        plt.plot(F_true[:-1], label = 'true tensor')
        plt.plot(F_predicted[:-1], label = 'predicted tensor')
        plt.legend()
        return F_true, F_predicted
    except:
        pass # to not cause errors when submitting

run_experiment(distance = 0.75, i = 41801)

In [ ]:
run_experiment(distance = 0.75, i = 24099)

In [ ]:
def FlattenMatrix(M, num_digits=8):
    '''Convenience function to write CSV files.'''
    return ' '.join([f'{v:.{num_digits}e}' for v in M.flatten()])

        
def find_fundanental_matrix(batch_id, image_1_id, image_2_id):
    
    img1 = cv.imread(f'{src}/test_images/{batch_id}/{image_1_id}.png',0)  #queryimage # left image
    img2 = cv.imread(f'{src}/test_images/{batch_id}/{image_2_id}.png',0) #trainimage # right image
    sift = cv.SIFT_create()
    # find the keypoints and descriptors with SIFT
    kp1, des1 = sift.detectAndCompute(img1,None)
    kp2, des2 = sift.detectAndCompute(img2,None)
    # FLANN parameters
    FLANN_INDEX_KDTREE = 1
    index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 220)
    search_params = dict(checks=220)
    flann = cv.FlannBasedMatcher(index_params,search_params)
    matches = flann.knnMatch(des1,des2,k=2)
    pts1 = []
    pts2 = []
    # ratio test as per Lowe's paper
    for i,(m,n) in enumerate(matches):
        if m.distance < 0.75*n.distance:
            pts2.append(kp2[m.trainIdx].pt)
            pts1.append(kp1[m.queryIdx].pt)

    pts1 = np.int32(pts1)
    pts2 = np.int32(pts2)
    F, mask = cv.findFundamentalMat(pts1,pts2,cv.FM_LMEDS)
    return F

In [ ]:
F_dict = {}

for i, row in enumerate(test_samples):
    sample_id, batch_id, image_1_id, image_2_id = row
    #F = np.zeros((3, 3))
    try:
        F = find_fundanental_matrix(batch_id, image_1_id, image_2_id)
        if len(F.flatten()) != 9:
            F = np.zeros((3, 3)) # Notebook Threw Exception while scoring
    except:
        F = np.zeros((3, 3))
     
    #F_dict[sample_id] = FlattenMatrix(F)
    F_dict[sample_id] = F
    #print( FlattenMatrix(F) )

In [ ]:
with open('submission.csv', 'w') as f:
    f.write('sample_id,fundamental_matrix\n')
    for sample_id, F in F_dict.items():
        f.write(f'{sample_id},{FlattenMatrix(F)}\n')
        

In [ ]:
#import pandas as pd
#pd.read_csv('submission.csv')